<a href="https://colab.research.google.com/github/ddhackiisc/code/blob/master/DILI/DILI_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install padelpy

In [4]:
import pandas as pd
import numpy as np
from padelpy import from_smiles

In [70]:
!rm -r /content/openbabel-openbabel-2-4-0.tar.gz

In [49]:
from google.colab import files
uploaded = files.upload()

Saving openbabel-openbabel-2-4-0.tar.gz to openbabel-openbabel-2-4-0.tar.gz


In [48]:
train_df = pd.read_csv("combined_train.csv")
train_smiles = train_df['smiles'].to_numpy()
train_labels = train_df['label'].to_numpy()


In [45]:
def create_bit_vector(fingerprints):
  bit_vector = np.empty((881))
  for j in range(881):
    bit_vector[j] = int(fingerprints['PubchemFP'+str(j)])
  return bit_vector


def pubchem_fingerprints(smiles_array):
  fingerprints = np.empty((len(smiles_array), 881))
  for i in range(len(smiles_array)):
    print("Processing Molecule {}".format(i))
    pubc_fingerp = from_smiles(smiles_array[i], descriptors=False, fingerprints=True, timeout=30)
    pubc_fingerp_b = create_bit_vector(pubc_fingerp)
    fingerprints[i,:] = pubc_fingerp_b
  return fingerprints

In [46]:
train_input = pubchem_fingerprints(train_smiles)

Processing Molecule 0
Processing Molecule 1
Processing Molecule 2
Processing Molecule 3
Processing Molecule 4
Processing Molecule 5
Processing Molecule 6
Processing Molecule 7
Processing Molecule 8
Processing Molecule 9
Processing Molecule 10
Processing Molecule 11
Processing Molecule 12
Processing Molecule 13
Processing Molecule 14
Processing Molecule 15
Processing Molecule 16
Processing Molecule 17
Processing Molecule 18
Processing Molecule 19
Processing Molecule 20
Processing Molecule 21
Processing Molecule 22
Processing Molecule 23
Processing Molecule 24
Processing Molecule 25
Processing Molecule 26
Processing Molecule 27
Processing Molecule 28
Processing Molecule 29
Processing Molecule 30
Processing Molecule 31
Processing Molecule 32
Processing Molecule 33
Processing Molecule 34
Processing Molecule 35
Processing Molecule 36
Processing Molecule 37
Processing Molecule 38
Processing Molecule 39
Processing Molecule 40
Processing Molecule 41
Processing Molecule 42
Processing Molecule 4